# Mini Proyecto

datos: https://www.cs.cmu.edu/~ark/personas/

In [2]:
# Librerias

import pandas as pd
import pyspark
# Cargar Funciones Extra
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Proyecto_1').getOrCreate()

22/02/26 23:21:01 WARN Utils: Your hostname, MacBook-Air-de-Daniel.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
22/02/26 23:21:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/26 23:21:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:
#Carga de bibliotecas extras
import numpy as np
import matplotlib.pyplot as plt

#Carga de NLTK
import nltk
import nltk
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielfragoso/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def read_tsv(direccion):
    import csv
    tsv_file = open(direccion)
    read_tsv = csv.reader(tsv_file, delimiter= "\t")
    lista = [row for row in read_tsv]
    tsv_file.close()
    return(spark.sparkContext.parallelize(lista))

def read_txt(direccion):
    import csv
    txt_file = open(direccion)
    read_tsv = csv.reader(txt_file, delimiter= "\t")
    lista = [row for row in read_tsv]
    txt_file.close()
    return(spark.sparkContext.parallelize(lista))

In [5]:
# Importamos datos:

character = read_tsv('MovieSummaries/character.metadata.tsv')
movie = read_tsv('MovieSummaries/movie.metadata.tsv')

In [6]:
name = read_txt('MovieSummaries/name.clusters.txt')
tvtropes = read_txt('MovieSummaries/tvtropes.clusters.txt')
plot_summaries =read_txt('MovieSummaries/plot_summaries.txt')

¿Cual es género más visto?

In [7]:
def sep_gen(renglon):
    renglon_separado = renglon[8]
    import re
    generos = re.findall(r'"([\w\s]+)"', renglon_separado)
    for g in generos:
        yield g

In [8]:
genere = movie.flatMap(sep_gen)
genere = genere.map(lambda renglon: (renglon, 1))
# Conteo
genere_count = genere.reduceByKey(lambda val_1 , val_2: val_1 + val_2).sortBy(lambda renglon: renglon[1], ascending = False)

22/02/26 23:21:23 WARN TaskSetManager: Stage 0 contains a task of very large size (4397 KiB). The maximum recommended task size is 1000 KiB.


In [9]:
genere_count.take(10)

[('Drama', 34007),
 ('Comedy', 16349),
 ('Romance Film', 10234),
 ('Action', 8798),
 ('Thriller', 8744),
 ('Short Film', 8141),
 ('World cinema', 7155),
 ('Crime Fiction', 6948),
 ('Indie', 6897),
 ('Documentary', 5630)]

La palabra más larga de las descripciones

In [89]:
def com_string(x, y):
    if len(x) > len(y):
        return x
    return y

def del_url(renglon):
    import re
    generos = re.sub(r'http\S+', ' ', renglon)
    return generos

In [90]:
desc = plot_summaries.map(lambda x: x[1])
desc = desc.map(del_url)

In [91]:
# Reduzcammos a palabras

words = desc.map(lambda x: x.split(' '))

#Eliminamos signos de puntuación y espacios en blanco
import string
def removePunctuationsFunct(x):
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    return filtered

#Removemos stopwords (palabras vacias)
def removeStopWordsFunct(x):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence

words = words.map(removeStopWordsFunct)
words = words.map(removePunctuationsFunct)


In [92]:
def sep(renglon):
    for g in renglon:
        return g

words_sep = words.map(sep)

In [93]:
word_legth = words_sep.reduce(com_string)

22/02/27 16:41:31 WARN TaskSetManager: Stage 42 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


In [94]:
word_legth

'PlotdatePrologueThursdayMondaySaturdayTuesdayFridaySundayWednesdayEpilogue'

Contar el número de documentos en los que ocurre cada palabra, bigrama y trigrama.

In [103]:
cont = plot_summaries.map(lambda x: (x[0], x[1].count('war')))

In [107]:
cont = cont.sortBy(lambda renglon: renglon[1], ascending = False)

22/02/27 18:10:22 WARN TaskSetManager: Stage 47 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.
22/02/27 18:10:23 WARN TaskSetManager: Stage 48 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


In [108]:
cont.take(5)

22/02/27 18:10:32 WARN TaskSetManager: Stage 49 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


[('2572919', 87),
 ('8885378', 34),
 ('2856257', 33),
 ('302970', 32),
 ('5579906', 31)]